<a href="https://colab.research.google.com/github/anqi-guo/anomaly_detection_isolation_forest/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 读取数据

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import os

path = '/content/drive/My Drive/work/在线监测/data'
data = pd.read_csv(f'{path}/unprocessed_data.csv')
data.info(memory_usage=True)

Mounted at /content/drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8104103 entries, 0 to 8104102
Data columns (total 14 columns):
 #   Column      Dtype  
---  ------      -----  
 0   排口ID        object 
 1   污染物ID       object 
 2   监测时间        object 
 3   污染物浓度值      float64
 4   污染物排放量      float64
 5   排放量         float64
 6   流量          float64
 7   污染物浓度上限值    float64
 8   是否生产        int64  
 9   生产状态_流量中位数  float64
 10  企业名称        object 
 11  城市名称        object 
 12  区县名称        object 
 13  是否异常        float64
dtypes: float64(7), int64(1), object(6)
memory usage: 865.6+ MB


In [ ]:
data['监测时间'] = pd.to_datetime(data['监测时间'])
data['是否异常'] = data['是否异常'].astype('category')

for c in data.select_dtypes(include=['object','int64']).columns:
  data[c] = data[c].astype('category')

for c in data.select_dtypes(include=['float64']).columns:
  data[c] = data[c].astype('float32')

data.info(memory_usage=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8104103 entries, 0 to 8104102
Data columns (total 14 columns):
 #   Column      Dtype         
---  ------      -----         
 0   排口ID        category      
 1   污染物ID       category      
 2   监测时间        datetime64[ns]
 3   污染物浓度值      float32       
 4   污染物排放量      float32       
 5   排放量         float32       
 6   流量          float32       
 7   污染物浓度上限值    float32       
 8   是否生产        category      
 9   生产状态_流量中位数  float32       
 10  企业名称        category      
 11  城市名称        category      
 12  区县名称        category      
 13  是否异常        category      
dtypes: category(7), datetime64[ns](1), float32(6)
memory usage: 316.9 MB


In [ ]:
import datetime
data = data[data['监测时间'] > datetime.datetime(2021,10,1)]

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
tqdm.pandas()

big_dict = {}
for c in tqdm(data.select_dtypes(include=['category']).columns):
  le = LabelEncoder()
  data[c] = le.fit_transform(data[c])
  big_dict[c] = list(le.classes_)

  0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
big_dict['是否生产'] = [0.0, 1.0]

In [ ]:
import json
with open(f'{path}/lable_encoder.json', 'w', encoding='utf-8') as outfile:
    json.dump(big_dict, outfile, ensure_ascii=False, indent=4)

In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

data.sort_values(by=['排口ID','污染物ID','监测时间'], inplace=True)

# diff features
for i in tqdm(range(1,4), desc='diff'):
  data[f'pol_diff{i}'] = data.groupby(['排口ID','污染物ID'])['污染物浓度值'].diff(i)
  data[f'pfl_diff{i}'] = data.groupby(['排口ID','污染物ID'])['排放量'].diff(i)
  data[f'll_diff{i}'] = data.groupby(['排口ID','污染物ID'])['流量'].diff(i)

# pct_chg features
for i in tqdm(range(1,4), desc='pct_chg'):
  data[f'pol_pctchg{i}'] = data.groupby(['排口ID','污染物ID'])['污染物浓度值'].pct_change(i)
  data[f'pfl_pctchg{i}'] = data.groupby(['排口ID','污染物ID'])['排放量'].pct_change(i)
  data[f'll_pctchg{i}'] = data.groupby(['排口ID','污染物ID'])['流量'].pct_change(i)

# lag features
for i in tqdm(range(1,4), desc='lag'):
  data[f'pol_lag{i}'] = data.groupby(['排口ID','污染物ID'])['污染物浓度值'].shift(i)
  data[f'pfl_lag{i}'] = data.groupby(['排口ID','污染物ID'])['排放量'].shift(i)
  data[f'll_lag{i}'] = data.groupby(['排口ID','污染物ID'])['流量'].shift(i)
  data[f'pol_diff_lag{i}'] = data.groupby(['排口ID','污染物ID'])[f'pol_diff{i}'].shift(i)
  data[f'pol_pctchg_lag{i}'] = data.groupby(['排口ID','污染物ID'])[f'pol_pctchg{i}'].shift(i)
  data[f'pfl_diff_lag{i}'] = data.groupby(['排口ID','污染物ID'])[f'pfl_diff{i}'].shift(i)
  data[f'pfl_pctchg_lag{i}'] = data.groupby(['排口ID','污染物ID'])[f'pfl_pctchg{i}'].shift(i)
  data[f'll_diff_lag{i}'] = data.groupby(['排口ID','污染物ID'])[f'll_diff{i}'].shift(i)
  data[f'll_pctchg_lag{i}'] = data.groupby(['排口ID','污染物ID'])[f'll_pctchg{i}'].shift(i)

diff:   0%|          | 0/3 [00:00<?, ?it/s]

pct_chg:   0%|          | 0/3 [00:00<?, ?it/s]

lag:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# 时间特征
data['year'] = data['监测时间'].dt.year
data['month'] = data['监测时间'].dt.month
data['day'] = data['监测时间'].dt.day
data['hour'] = data['监测时间'].dt.hour

del data['监测时间']

In [ ]:
import gc
gc.collect()

for c in data.select_dtypes(include=['float64']):
  data[c] = data[c].astype('float32')

In [ ]:
pd.set_option('display.max_columns', None)
data.head()

,排口ID,污染物ID,污染物浓度值,污染物排放量,排放量,流量,污染物浓度上限值,是否生产,生产状态_流量中位数,企业名称,城市名称,区县名称,是否异常,pol_diff1,pfl_diff1,ll_diff1,pol_diff2,pfl_diff2,ll_diff2,pol_diff3,pfl_diff3,ll_diff3,pol_pctchg1,pfl_pctchg1,ll_pctchg1,pol_pctchg2,pfl_pctchg2,ll_pctchg2,pol_pctchg3,pfl_pctchg3,ll_pctchg3,pol_lag1,pfl_lag1,ll_lag1,pol_diff_lag1,pol_pctchg_lag1,pfl_diff_lag1,pfl_pctchg_lag1,ll_diff_lag1,ll_pctchg_lag1,pol_lag2,pfl_lag2,ll_lag2,pol_diff_lag2,pol_pctchg_lag2,pfl_diff_lag2,pfl_pctchg_lag2,ll_diff_lag2,ll_pctchg_lag2,pol_lag3,pfl_lag3,ll_lag3,pol_diff_lag3,pol_pctchg_lag3,pfl_diff_lag3,pfl_pctchg_lag3,ll_diff_lag3,ll_pctchg_lag3,year,month,day,hour
4104,0,11,6.919,0.0,269.393005,74.831001,9.0,1,66.266998,4,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,10,1,1
4105,0,11,6.896,0.0,266.433990,74.009003,9.0,1,66.266998,4,5,0,0,-0.023,-2.959015,-0.821999,NaN,NaN,NaN,NaN,NaN,NaN,-0.003324,-0.010984,-0.010985,NaN,NaN,NaN,NaN,NaN,NaN,6.919,269.393005,74.831001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,10,1,2
4106,0,11,6.933,0.0,260.334015,72.315002,9.0,1,66.266998,4,5,0,0,0.037,-6.099976,-1.694000,0.014,-9.058990,-2.515999,NaN,NaN,NaN,0.005365,-0.022895,-0.022889,0.002023,-0.033627,-0.033622,NaN,NaN,NaN,6.896,266.433990,74.009003,-0.023,-0.003324,-2.959015,-0.010984,-0.821999,-0.010985,6.919,269.393005,74.831001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021,10,1,3
4107,0,11,6.894,0.0,266.026001,73.896004,9.0,1,66.266998,4,5,0,0,-0.039,5.691986,1.581001,-0.002,-0.407990,-0.112999,-0.025,-3.367004,-0.934998,-0.005625,0.021864,0.021863,-0.000290,-0.001531,-0.001527,-0.003613,-0.012498,-0.012495,6.933,260.334015,72.315002,0.037,0.005365,-6.099976,-0.022895,-1.694000,-0.022889,6.896,266.433990,74.009003,NaN,NaN,NaN,NaN,NaN,NaN,6.919,269.393005,74.831001,NaN,NaN,NaN,NaN,NaN,NaN,2021,10,1,4
4108,0,11,6.904,0.0,246.242996,68.401001,9.0,1,66.266998,4,5,0,0,0.010,-19.783005,-5.495003,-0.029,-14.091019,-3.914001,0.008,-20.190994,-5.608002,0.001451,-0.074365,-0.074361,-0.004183,-0.054127,-0.054124,0.001160,-0.075782,-0.075775,6.894,266.026001,73.896004,-0.039,-0.005625,5.691986,0.021864,1.581001,0.021863,6.933,260.334015,72.315002,0.014,0.002023,-9.05899,-0.033627,-2.515999,-0.033622,6.896,266.433990,74.009003,NaN,NaN,NaN,NaN,NaN,NaN,2021,10,1,5


In [ ]:
data.shape

(4882885, 62)

In [ ]:
data = data[data['pol_lag3'].notnull()]

data.shape

(4879543, 62)

In [ ]:
data.to_csv(f'{path}/encoded_data.csv', index=False)